# Хакатон Skillfactory x Teplitsa Lab

## Задача 1. Деньги государства у НКО

Есть дамп данных о всех НКО в России, содержится информация о получении государственных грантов, госконтрактов и субсидий, регионе и дате регистрации и ОКВЭД (классификатор экономической деятельности).

- Проверить, есть ли зависимость вероятности получения грантов от государства / госконтрактов от региона регистрации организации
- Проверить, есть ли зависимость вероятности получения грантов от государства / госконтрактов от возраста организации
- Проверить, есть ли зависимость вероятности получения грантов от государства / госконтрактов от экономической деятельности
- Любые дополнительные выводы и проверки

### Данные

Источник данных: проект [Открытые НКО](https://openngo.ru/)

[Дамп данных в формате JSON](https://drive.google.com/drive/folders/1WiGeZPnoiBqgeQL59AkEC4JiszFNAWl9?usp=sharing), дата актуальности: 26.08.2021

[Описание полей](https://github.com/infoculture/openngo-data-reference/wiki/%D0%A5%D0%B0%D1%80%D0%B0%D0%BA%D1%82%D0%B5%D1%80%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B8-%D0%B8-%D1%80%D0%B0%D1%81%D1%88%D0%B8%D1%84%D1%80%D0%BE%D0%B2%D0%BA%D0%B8-%D0%BE%D1%82%D0%BA%D1%80%D1%8B%D1%82%D1%8B%D1%85-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85) на гитхабе

UPD: [foundations_codes.csv](https://github.com/Teplitsa/CSRLab/blob/main/Skillfactory%20x%20Teplitsa%20Lab/foundations_codes.csv) - таблица с более точными кодами правовой формы (ОКОПФ) для фондов всех видов

## 1. Первичная обработка данных

In [1]:
# работа с данными
import pandas as pd
import numpy as np
import re

# библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

%matplotlib inline
rcParams['figure.figsize'] = 12, 6

# библиотека для ускорения pandas
import swifter

# библиотека для расширенной работы с датами
from dateutil.parser import parse

In [2]:
# для работы с предупреждениями
import warnings  
# игнорирование предупреждений
warnings.filterwarnings('ignore')

In [3]:
# вывод всех значений ячейки
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [4]:
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
 
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

In [5]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [6]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [7]:
nko_data = pd.read_csv('data/nko_dropped.csv')

In [8]:
nko_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747034 entries, 0 to 747033
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          747034 non-null  int64  
 1   regionName          747009 non-null  object 
 2   fullName            747034 non-null  object 
 3   dateReg             321416 non-null  object 
 4   minjustForm         461073 non-null  object 
 5   ogrn                747034 non-null  int64  
 6   opf                 747034 non-null  object 
 7   egrulStatus         747034 non-null  object 
 8   mainOkved           747034 non-null  object 
 9   regionCode          747034 non-null  int64  
 10  incomeTotal         747034 non-null  float64
 11  inn                 744430 non-null  float64
 12  originDate          747034 non-null  object 
 13  dateLiquid          399961 non-null  object 
 14  address             747034 non-null  object 
 15  okogu               747034 non-nul

In [9]:
nko_data.sample()

,Unnamed: 0,regionName,fullName,dateReg,minjustForm,ogrn,opf,egrulStatus,mainOkved,regionCode,incomeTotal,inn,originDate,dateLiquid,address,okogu,hasRegionalSupport,addOkved,dateOgrn
429262,429262,Томская область,"ГАРАЖНЫЙ КООПЕРАТИВ ""БЕРЕЗКА""",NaN,NaN,1087017022949,"{'name': 'Гаражные и гаражно-строительные кооперативы', 'code': '20101', 'version': 'okopf'}",Действует,"{'name': 'Управление эксплуатацией нежилого фонда за вознаграждение или на договорной основе', 'code': '68.32.2', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}",70,0.0,7.017220e+09,{'$date': '2008-08-13T00:00:00.000Z'},NaN,"634040, ОБЛАСТЬ ТОМСКАЯ, ГОРОД ТОМСК, ТРАКТ ИРКУТСКИЙ, ДОМ 150, КВ./ОФИС 69","{'name': 'Организации, учрежденные юридическими лицами или гражданами, или юридическими лицами и гражданами совместно', 'code': '4210014'}",False,[],{'$date': '2008-08-13T00:00:00.000Z'}


In [10]:
# список признаков, которые необходимо преобразовать
list_to_dir = ['opf', 'mainOkved', 'okogu', 'addOkved']

# преобразуем строки в словари
for series in list_to_dir:
    nko_data[series] = nko_data[series].swifter.apply(lambda x: eval(x))

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [11]:
# список признаков, которые необходимо преобразовать
list_to_datatime = ['dateReg', 'originDate', 'dateLiquid', 'dateOgrn']

# преобразуем строки в дату и время
for series in list_to_datatime:
    # замена пропусков и приводим в строковому типу
    nko_data[series].fillna('not data', inplace=True)
    nko_data[series] = nko_data[series].astype('string')
    # выделение даты из строки
    nko_data[series] = nko_data[series].swifter.apply(
        lambda x: parse(x, fuzzy=True) if x != 'not data' else x)

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

In [12]:
# разбираем признак opf на составляющие
nko_data['opf_name'] = nko_data['opf'].swifter.apply(lambda x: x['name'])
nko_data['opf_code'] = nko_data['opf'].swifter.apply(lambda x: x['code'])
# удаляем признак
nko_data.drop(['opf'], axis=1, inplace=True)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [13]:
# разбираем признак mainOkved на составляющие
nko_data['mainOkved_name'] = nko_data['mainOkved'].swifter.apply(
    lambda x: x['name'])
nko_data['mainOkved_code'] = nko_data['mainOkved'].swifter.apply(
    lambda x: x['code'])
# удаляем признак
nko_data.drop(['mainOkved'], axis=1, inplace=True)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [14]:
# добавим признак кол-во addOkved
nko_data['num_of_addOkved'] = nko_data['addOkved'].swifter.apply(lambda x: len(x))

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [15]:
# обработаем значения с помощью регулярных выражений
def index_from_address(row):
    '''Поиск индекса длиной от 5 до 6 цифр'''
    
    result = re.search(r'\d{5,6}', row)
    
    if result:
        return int(result[0])
    return None

In [16]:
# выделяем индекс из адреса в новый признак
nko_data['index_address'] = nko_data['address'].swifter.apply(
    index_from_address)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [17]:
nko_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747034 entries, 0 to 747033
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   Unnamed: 0          747034 non-null  int64                  
 1   regionName          747009 non-null  object                 
 2   fullName            747034 non-null  object                 
 3   dateReg             747034 non-null  object                 
 4   minjustForm         461073 non-null  object                 
 5   ogrn                747034 non-null  int64                  
 6   egrulStatus         747034 non-null  object                 
 7   regionCode          747034 non-null  int64                  
 8   incomeTotal         747034 non-null  float64                
 9   inn                 744430 non-null  float64                
 10  originDate          747034 non-null  datetime64[ns, tzutc()]
 11  dateLiquid          747034

In [18]:
# посмотрим количество пропусков в каждом признаке
nko_data.isna().sum()

Unnamed: 0                 0
regionName                25
fullName                   0
dateReg                    0
minjustForm           285961
ogrn                       0
egrulStatus                0
regionCode                 0
incomeTotal                0
inn                     2604
originDate                 0
dateLiquid                 0
address                    0
okogu                      0
hasRegionalSupport         0
addOkved                   0
dateOgrn                   0
opf_name                 415
opf_code               45209
mainOkved_name        204681
mainOkved_code        204681
num_of_addOkved            0
index_address           9725
dtype: int64

#### Проверим региональные признаки

In [19]:
nko_data['regionName'].value_counts()

Москва                          80178
Московская область              37927
Санкт-Петербург                 31767
Краснодарский край              23337
Свердловская область            23030
                                ...  
Республика Ингушетия             1249
Еврейская автономная область     1197
Чукотский автономный округ        505
Ненецкий автономный округ         398
Не определено                      91
Name: regionName, Length: 86, dtype: int64

In [20]:
nko_data[(nko_data['regionName'] == 'Не определено')
         & (nko_data['regionCode'] == 99)]

,Unnamed: 0,regionName,fullName,dateReg,minjustForm,ogrn,egrulStatus,regionCode,incomeTotal,inn,originDate,dateLiquid,address,okogu,hasRegionalSupport,addOkved,dateOgrn,opf_name,opf_code,mainOkved_name,mainOkved_code,num_of_addOkved,index_address
3277,3277,Не определено,"КРЕДИТНЫЙ ПОТРЕБИТЕЛЬСКИЙ КООПЕРАТИВ ""БОЛЬШОЙ КАПИТАЛ""",not data,NaN,1209900000100,Действует,99,0.0,9.901037e+09,2020-12-28 00:00:00+00:00,not data,"468320, Г. БАЙКОНУР, УЛ. ГОРЬКОГО, ДОМ Д. 16","{'name': None, 'code': None}",False,"[{'name': 'Консультирование по вопросам коммерческой деятельности и управления', 'code': '70.22', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}, {'name': 'Деятельность по предоставлению консультационных услуг по вопросам финансового посредничества', 'code': '66.19.4', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}, {'name': 'Деятельность по предоставлению займов промышленности', 'code': '64.92.2', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}, {'name': 'Деятельность по предоставлению потребительского кредита', 'code': '64.92.1', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}, {'name': 'Деятельность по предоставлению денежных ссуд под залог недвижимого имущества', 'code': '64.92.3', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}, {'name': 'Деятельность по оказанию консультационных и информационных услуг', 'code': '63.99.1', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}]",2020-12-28 00:00:00+00:00,Кредитные потребительские кооперативы,20104,Деятельность по финансовой взаимопомощи,64.99.6,6,468320.0
3721,3721,Не определено,"БАЙКОНУРСКАЯ ГОРОДСКАЯ ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ ЗАЩИТЫ ЖИВОТНЫХ ""ПОДАРИ НАДЕЖДУ""",not data,Общественная организация,1205000106970,Действует,99,0.0,9.901037e+09,2020-11-20 00:00:00+00:00,not data,"468325, ГОРОД БАЙКОНУР, МИКРОРАЙОН 5-Й, ДОМ 25, КВАРТИРА 16А","{'name': None, 'code': None}",False,"[{'name': 'Предоставление прочих персональных услуг, не включенных в другие группировки', 'code': '96.09', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}, {'name': 'Деятельность ветеринарная', 'code': '75.00', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}]",2020-11-20 00:00:00+00:00,Общественные организации,20200,"Деятельность прочих общественных организаций, не включенных в другие группировки",94.99,2,468325.0
35939,35939,Не определено,"ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ ""БАЙКОНУРСКАЯ ФЕДЕРАЦИЯ БОКСА""",not data,Общественная организация,1095000003977,Ликвидирована,99,0.0,9.901034e+09,2009-07-30 00:00:00+00:00,2017-10-11 00:00:00+00:00,"468320, ГОРОД БАЙКОНУР, ГОРОД БАЙКОНУР, УЛИЦА АВИАЦИОННАЯ, ДОМ 9","{'name': None, 'code': None}",False,[],2009-07-30 00:00:00+00:00,ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ,None,"Деятельность прочих общественных организаций, не включенных в другие группировки",94.99,0,468320.0
53749,53749,Не определено,"ПОТРЕБИТЕЛЬСКИЙ КООПЕРАТИВ ""ГАРАЖНО-СТРОИТЕЛЬНЫЙ КООПЕАТИВ ""ДИНАМО""",not data,NaN,1039923001271,Ликвидирована,99,0.0,9.901006e+09,2003-07-22 00:00:00+00:00,2005-04-05 00:00:00+00:00,"468320, ГОРОД БАЙКОНУР, ПРОСПЕКТ АБАЯ, ДОМ 0","{'name': None, 'code': None}",False,[],2003-07-22 00:00:00+00:00,Потребительский кооператив,85,None,None,0,468320.0
53752,53752,Не определено,"ПОТРЕБИТЕЛЬСКИЙ КООПЕРАТИВ ""ГАРАЖНО-СТРОИТЕЛЬНЫЙ КООПЕРАТИВ № 3А""",2002-05-27 00:00:00+00:00,NaN,1039923000974,Ликвидирована,99,0.0,9.901006e+09,2002-05-27 00:00:00+00:00,2011-02-18 00:00:00+00:00,"468320, ГОРОД БАЙКОНУР, УЛИЦА ЯНГЕЛЯ, ДОМ 0","{'name': None, 'code': None}",False,"[{'name': 'Деятельность прочих общественных организаций, не включенных в другие группировки', 'code': '91.33', 'version': 'ОК 029-2001 (КДЕС Ред. 1)'}, {'name': 'Эксплуатация гаражей, стоянок для автотранспортных средств, велосипедов и т.п.', 'code': '63.21.24', 'version': 'ОК 029-2001 (КДЕС Ред. 1)'}]",2003-05-20 00:00:00+00:00,Потребительский кооператив,85,Хранение и складирование,63.12,2,468320.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712352,712352,Не определено,"ПРОФСОЮЗНАЯ ОРГАНИЗАЦИЯ РАБОТНИКОВ ГОСУДАРСТВЕННОГО УНИТАРНОГО ПРЕДПРИЯТИЯ ""ПРОИЗВОДСТВЕННО-ЭНЕРГ

Все "Не определено" - это г. Байконур, на территории Казахстана.

In [21]:
# заменяем Не определен на Казахстан
nko_data['regionName'] = nko_data['regionName'].swifter.apply(
    lambda x: x if x != 'Не определено' else 'Казахстан')

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [22]:
nko_data[nko_data['regionName'].isna()]

,Unnamed: 0,regionName,fullName,dateReg,minjustForm,ogrn,egrulStatus,regionCode,incomeTotal,inn,originDate,dateLiquid,address,okogu,hasRegionalSupport,addOkved,dateOgrn,opf_name,opf_code,mainOkved_name,mainOkved_code,num_of_addOkved,index_address
31530,31530,NaN,"ТОВАРИЩЕСТВО СОБСТВЕННИКОВ ЖИЛЬЯ ""КАРЛА МАРКСА, 53 ""А""",not data,Товарищество собственников жилья,1031621015722,Действует,0,0.0,1.655075e+09,2003-08-28 00:00:00+00:00,not data,"420015, РЕСПУБЛИКА Татарстан, УЛИЦА К.МАРКСА, ДОМ 53, КОРПУС А","{'name': 'Организации, учрежденные юридическими лицами или гражданами, или юридическими лицами и гражданами совместно', 'code': '4210014'}",False,[],2003-08-28 00:00:00+00:00,Товарищества собственников жилья,20716,None,None,0,420015.0
46886,46886,NaN,"ТОВАРИЩЕСТВО СОБСТВЕННИКОВ ЖИЛЬЯ ""КОМФОРТ""",not data,Товарищество собственников жилья,1041630207145,Ликвидирована,0,0.0,1.660071e+09,2004-04-08 00:00:00+00:00,not data,"420100, РЕСПУБЛИКА ТАТАРСТАН, УЛИЦА ПРОСПЕКТ ПОБЕДЫ, ДОМ 190, КВ./ОФИС 36","{'name': None, 'code': None}",False,"[{'name': 'Управление недвижимым имуществом', 'code': '70.32', 'version': 'ОК 029-2001 (КДЕС Ред. 1)'}, {'name': 'Производство общестроительных работ', 'code': '45.21', 'version': 'ОК 029-2001 (КДЕС Ред. 1)'}, {'name': 'Производство штукатурных работ', 'code': '45.41', 'version': 'ОК 029-2001 (КДЕС Ред. 1)'}, {'name': 'Предоставление различных видов услуг', 'code': '74.8', 'version': 'ОК 029-2001 (КДЕС Ред. 1)'}, {'name': 'Чистка и уборка производственных и жилых помещений и оборудования', 'code': '74.70.1', 'version': 'ОК 029-2001 (КДЕС Ред. 1)'}, {'name': 'Производство отделочных работ', 'code': '45.4', 'version': 'ОК 029-2001 (КДЕС Ред. 1)'}, {'name': 'Строительство зданий и сооружений', 'code': '45.2', 'version': 'ОК 029-2001 (КДЕС Ред. 1)'}, {'name': 'Монтаж инженерного оборудования зданий и сооружений', 'code': '45.3', 'version': 'ОК 029-2001 (КДЕС Ред. 1)'}, {'name': 'Деятельность органов местного самоуправления по управлению вопросами общего характера', 'code': '75.11.3', 'version': 'ОК 029-2001 (КДЕС Ред. 1)'}]",2004-04-08 00:00:00+00:00,Товарищества собственников жилья,20716,Разборка и снос зданий; производство земляных работ,45.11,9,420100.0
51173,51173,NaN,"ТОВАРИЩЕСТВО СОБСТВЕННИКОВ ЖИЛЬЯ ""КАРАТАУ""",1998-06-27 00:00:00+00:00,Товарищество собственников жилья,1031630200249,Ликвидирована,0,0.0,1.660035e+09,1998-06-27 00:00:00+00:00,2008-10-02 00:00:00+00:00,"420140, РЕСПУБЛИКА ТАТАРСТАН, УЛИЦА КАРАТАУ, ДОМ 32","{'name': None, 'code': None}",False,[],2003-01-05 00:00:00+00:00,Товарищество собственников жилья,94,None,None,0,420140.0
53316,53316,NaN,"ЖИЛИЩНО-СТРОИТЕЛЬНЫЙ КООПЕРАТИВ ""СПАРТАК-16""",1969-01-20 00:00:00+00:00,NaN,1031628201560,Ликвидирована,0,0.0,1.659016e+09,1969-01-20 00:00:00+00:00,2006-05-19 00:00:00+00:00,"420101, РЕСПУБЛИКА ТАТАРСТАН, УЛИЦА МАВЛЮТОВА, ДОМ 20","{'name': None, 'code': None}",False,[],2003-01-17 00:00:00+00:00,Потребительский кооператив,85,Управление эксплуатацией жилого фонда,70.32.1,0,420101.0
53323,53323,NaN,"ФОНД ""НОВЫЕ ТЕХНОЛОГИИ РАСПРОСТРАНЕНИЯ КУЛЬТУРЫ""",1994-09-27 00:00:00+00:00,NaN,1031630213581,Ликвидирована,0,0.0,1.660022e+09,1994-09-27 00:00:00+00:00,2003-07-29 00:00:00+00:00,"РЕСПУБЛИКА ТАТАРСТАН, УЛИЦА Н.ЕРШОВА, ДОМ 28, КОРПУС А","{'name': None, 'code': None}",False,[],2003-07-18 00:00:00+00:00,Фонд,88,None,None,0,NaN
53841,53841,NaN,"ЖИЛИЩНО-СТРОИТЕЛЬНЫЙ КООПЕРАТИВ ""СПАРТАК - 2""",1997-07-03 00:00:00+00:00,NaN,1031622502262,Ликвидирована,0,0.0,1.654022e+09,1997-07-03 00:00:00+00:00,2006-08-18 00:00:00+00:00,"420021, РЕСПУБЛИКА ТАТАРСТАН, УЛИЦА КАМАЛА ГАЛИАСКАРА, ДОМ 47","{'name': None, 'code': None}",False,[],2003-01-10 00:00:00+00:00,Потребительский кооператив,85,None,None,0,420021.0
54766,54766,NaN,"КРЕДИТНЫЙ ПОТРЕБИТЕЛЬСКИЙ КООПЕРАТИВ ГРАЖДАН ""КРЕДИТНО-ИПОТЕЧНЫЙ СОЮЗ""",not data,NaN,1031625401565,Ликвидирована,0,0.0,1.657043e+09,2003-02-05 00:00:00+00:00,2009-03-24 00:00:00+00:00,"420044, РЕСПУБЛИКА ТАТАРСТАН, УЛИЦА ЯМАШЕВА, ДОМ 36","{'na

Все пропущенные значения относятся к Республике Татарстан с кодом региона - 16

In [23]:
nko_data[nko_data['regionCode'] == 16]

,Unnamed: 0,regionName,fullName,dateReg,minjustForm,ogrn,egrulStatus,regionCode,incomeTotal,inn,originDate,dateLiquid,address,okogu,hasRegionalSupport,addOkved,dateOgrn,opf_name,opf_code,mainOkved_name,mainOkved_code,num_of_addOkved,index_address
269,269,Республика Татарстан,МЕСТНАЯ МУСУЛЬМАНСКАЯ РЕЛИГИОЗНАЯ ОРГАНИЗАЦИЯ-ПРИХОД МЕЧЕТИ Д. АЗЯКУЛЬ МУХТАСИБАТА АКТАНЫШСКОГО РАЙОНА ЦЕНТРАЛИЗОВАННОЙ РЕЛИГИОЗНОЙ ОРГАНИЗАЦИИ-ДУХОВНОГО УПРАВЛЕНИЯ МУСУЛЬМАН РЕСПУБЛИКИ ТАТАРСТАН,not data,Религиозная организация,1201600046515,Действует,16,0.0,1.604011e+09,2020-07-08 00:00:00+00:00,not data,"423740, РЕСПУБЛИКА ТАТАРСТАН, РАЙОН АКТАНЫШСКИЙ, ДЕРЕВНЯ АЗЯКУЛЬ, УЛИЦА НАБЕРЕЖНАЯ, ДОМ 36","{'name': None, 'code': None}",False,[],2020-07-08 00:00:00+00:00,Религиозные организации,71500,Деятельность религиозных организаций,94.91,0,423740.0
270,270,Республика Татарстан,"МЕСТНАЯ МУСУЛЬМАНСКАЯ РЕЛИГИОЗНАЯ ОРГАНИЗАЦИЯ-ПРИХОД МЕЧЕТИ ""АХТАМ"" МУХТАСИБАТА СОВЕТСКОГО РАЙОНА ГОРОДА КАЗАНИ ЦЕНТРАЛИЗОВАННОЙ РЕЛИГИОЗНОЙ ОРГАНИЗАЦИИ - ДУХОВНОГО УПРАВЛЕНИЯ МУСУЛЬМАН РЕСПУБЛИКИ ТАТАРСТАН",not data,Религиозная организация,1201600046504,Действует,16,0.0,1.660347e+09,2020-07-08 00:00:00+00:00,not data,"420075, РЕСПУБЛИКА ТАТАРСТАН, ГОРОД КАЗАНЬ, УЛИЦА АСЫЛЪЯР, ДОМ 41 А","{'name': None, 'code': None}",False,[],2020-07-08 00:00:00+00:00,Религиозные организации,71500,Деятельность религиозных организаций,94.91,0,420075.0
273,273,Республика Татарстан,МЕСТНАЯ СТАРООБРЯДЧЕСКАЯ РЕЛИГИОЗНАЯ ОРГАНИЗАЦИЯ - ДАВЛИКЕЕВО-ВВЕДЕНСКАЯ СТАРООБРЯДЧЕСКАЯ ОБЩИНА СТАРО-ПОМОРСКОГО СОГЛАСИЯ ГОРОДА КАЗАНИ,not data,Религиозная организация,1201600025307,Действует,16,0.0,1.655436e+09,2020-03-27 00:00:00+00:00,not data,"420021, РЕСПУБЛИКА ТАТАРСТАН, ГОРОД КАЗАНЬ, УЛИЦА МАРТЫНА МЕЖЛАУКА, ДОМ 28, ЛИТЕР 1,2 ПОМЕЩЕНИЕ 1","{'name': None, 'code': None}",False,[],2020-03-27 00:00:00+00:00,Религиозные организации,71500,Деятельность религиозных организаций,94.91,0,420021.0
391,391,Республика Татарстан,"РЕГИОНАЛЬНОЕ ЭКОЛОГИЧЕСКОЕ ОБЩЕСТВЕННОЕ ДВИЖЕНИЕ РЕСПУБЛИКИ ТАТАРСТАН ""НАРОДНАЯ ИНИЦИАТИВА""",not data,Общественное движение,1201600046482,Действует,16,0.0,1.650393e+09,2020-07-08 00:00:00+00:00,not data,"423810, РЕСПУБЛИКА ТАТАРСТАН, ГОРОД НАБЕРЕЖНЫЕ ЧЕЛНЫ, УЛИЦА АКАДЕМИКА РУБАНЕНКО, ДОМ 12, ОФИС 305","{'name': None, 'code': None}",False,"[{'name': 'Деятельность в области права', 'code': '69.10', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}, {'name': 'Издание газет', 'code': '58.13', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}, {'name': 'Издание журналов и периодических изданий', 'code': '58.14', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}, {'name': 'Виды издательской деятельности прочие', 'code': '58.19', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}, {'name': 'Предоставление консультационных услуг в области экологии', 'code': '74.90.5', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}, {'name': 'Деятельность по охране исторических мест и зданий, памятников культуры', 'code': '91.03', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}, {'name': 'Деятельность по мониторингу загрязнения окружающей среды для физических и юридических лиц', 'code': '71.12.53', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}]",2020-07-08 00:00:00+00:00,Общественные движения,20210,"Деятельность прочих общественных организаций, не включенных в другие группировки",94.99,7,423810.0
392,392,Республика Татарстан,"ЧАСТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ ""ЛЮБИМАЯ ШКОЛА""",not data,Учреждение,1201600046471,Действует,16,0.0,1.659210e+09,2020-07-08 00:00:00+00:00,not data,"420064, РЕСПУБЛИКА ТАТАРСТАН, ГОРОД КАЗАНЬ, УЛИЦА ГАРИФА АХУНОВА, ДОМ 20, КВАРТИРА 124","{'name': None, 'code': None}",False,"[{'name': 'Образование дополнительное детей и взрослых', 'code': '85.41', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}, {'name': 'Образование основное общее', 'code': '85.13', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}, {'name': 'Образование среднее общее', 'code': '85.14', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}]",2020-07-08 00:00:00+00:00,Частные учреждения,75500,Образование начальное общее,85.12,3

In [27]:
# заменим пропуски в названии региона и в коде региона
nko_data['regionName'].fillna('Республика Татарстан', inplace=True)
nko_data.loc[nko_data['regionName'] ==
             'Республика Татарстан', 'regionCode'] = 16

Посмотрим признак код региона на наличие нулевых значений

In [28]:
nko_data['regionCode'].value_counts()

77    80178
50    37927
78    31767
23    23337
66    23030
      ...  
80      105
99       91
81       84
88       65
82       31
Name: regionCode, Length: 92, dtype: int64

In [36]:
# разбираем признак okogu на составляющие
nko_data['okogu_name'] = nko_data['okogu'].swifter.apply(lambda x: x['name'])
nko_data['okogu_code'] = nko_data['okogu'].swifter.apply(lambda x: x['code'])
# удаляем признак
nko_data.drop(['okogu'], axis=1, inplace=True)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [37]:
# создаем целевую переменную
nko_data['grant'] = nko_data['incomeTotal'].swifter.apply(
    lambda x: 1 if x > 0 else 0)

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

#### Работа с датами

In [47]:
# создаем признак, дата регистрации для всех организаций
nko_data['data_reg'] = nko_data['dateOgrn']
nko_data['data_reg'] = nko_data['dateReg'].where(
    nko_data['data_reg'] == 'not data', nko_data['data_reg'])

In [49]:
# исходные признаки делаем флагами, для организаций зарегистрированных до 2002 года и после
nko_data['dateOgrn'] = nko_data['dateOgrn'].swifter.apply(
    lambda x: 1 if x != 'not data' else 0)
nko_data['dateReg'] = nko_data['dateReg'].swifter.apply(
    lambda x: 1 if x != 'not data' else 0)

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [51]:
nko_data.sample(2)

,Unnamed: 0,regionName,fullName,dateReg,minjustForm,ogrn,egrulStatus,regionCode,incomeTotal,inn,originDate,dateLiquid,address,hasRegionalSupport,addOkved,dateOgrn,opf_name,opf_code,mainOkved_name,mainOkved_code,num_of_addOkved,index_address,okogu_name,okogu_code,grant,data_reg
49977,49977,Волгоградская область,"ПЕРВИЧНАЯ ПРОФСОЮЗНАЯ ОРГАНИЗАЦИЯ СПК ""СЕРГИЕВСКИЙ"" ДАНИЛОВСКОГО РАЙОНА ВОЛГОГРАДСКОЙ ТЕРРИТОРИАЛЬНОЙ (ОБЛ.) ОРГАНИЗАЦИИ ПРОФСОЮЗА РАБОТНИКОВ АПК РФ",1,Профессиональный союз,1103400004366,Ликвидирована,34,0.0,3.404003e+09,1999-06-22 00:00:00+00:00,2010-12-30 00:00:00+00:00,"403333, ОБЛАСТЬ ВОЛГОГРАДСКАЯ, РАЙОН ДАНИЛОВСКИЙ, СТАНИЦА СЕРГИЕВСКАЯ",False,[],1,ПРОФСОЮЗНАЯ ОРГАНИЗАЦИЯ,None,None,None,0,403333.0,None,None,0,2010-12-30 00:00:00+00:00
263094,263094,Приморский край,"ТОВАРИЩЕСТВО СОБСТВЕННИКОВ НЕДВИЖИМОСТИ ""СОГЛАСИЕ""",0,NaN,1182536002791,Действует,25,0.0,2.507013e+09,2018-02-01 00:00:00+00:00,not data,"692038, КРАЙ ПРИМОРСКИЙ, ГОРОД ЛЕСОЗАВОДСК, УЛИЦА ЛЕНИНСКАЯ, ДОМ 41, КВ./ОФИС ПОМ 1",False,"[{'name': 'Деятельность по чистке и уборке жилых зданий и нежилых помещений прочая', 'code': '81.22', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}, {'name': 'Деятельность по чистке и уборке прочая', 'code': '81.29', 'version': 'ОК 029-2014 (КДЕС Ред. 2)'}]",1,Товарищества собственников недвижимости,20700,Управление недвижимым имуществом за вознаграждение или на договорной основе,68.32,2,692038.0,"Организации, учрежденные юридическими лицами или гражданами, или юридическими лицами и гражданами совместно",4210014,0,2018-02-01 00:00:00+00:00


In [53]:
# удаляем признаки
nko_data.drop(['inn', 'address', 'addOkved'], axis=1, inplace=True)

In [54]:
nko_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747034 entries, 0 to 747033
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   Unnamed: 0          747034 non-null  int64                  
 1   regionName          747034 non-null  object                 
 2   fullName            747034 non-null  object                 
 3   dateReg             747034 non-null  int64                  
 4   minjustForm         461073 non-null  object                 
 5   ogrn                747034 non-null  int64                  
 6   egrulStatus         747034 non-null  object                 
 7   regionCode          747034 non-null  int64                  
 8   incomeTotal         747034 non-null  float64                
 9   originDate          747034 non-null  datetime64[ns, tzutc()]
 10  dateLiquid          747034 non-null  object                 
 11  hasRegionalSupport  747034

#### Работа с признаком `opf_name`

In [77]:
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Общественные организации' if x == 'ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Товарищества собственников недвижимости' if x == 'Товарищество собственников жилья' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Некоммерческие партнерства' if x == 'Некоммерческое партнерство' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [79]:
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Автономные некоммерческие организации' if x == 'Автономная некоммерческая организация' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Фонды' if x == 'Фонд' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Садоводческие, огороднические или дачные некоммерческие товарищества' if x == 'Садоводческое, огородническое или дачное некоммерческое товарищество' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [81]:
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Потребительские организации' if x == 'Потребительский кооператив' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Потребительские организации' if x == 'Потребительские кооперативы' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Потребительские организации' if x == 'Потребительские общества' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [84]:
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Ассоциации (союзы)' if x == 'Ассоциация (союз)' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Профсоюзные организации' if x == 'ПРОФСОЮЗНАЯ ОРГАНИЗАЦИЯ' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Садоводческие, огороднические или дачные некоммерческие организации' if x == 'Садоводческие, огороднические или дачные некоммерческие товарищества' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Садоводческие, огороднические или дачные некоммерческие организации' if x == 'Садоводческие, огороднические или дачные некоммерческие партнерства' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Садоводческие, огороднические или дачные некоммерческие организации' if x == 'Садоводческие, огороднические или дачные потребительские кооперативы' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [86]:
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Сельскохозяйственные потребительские кооперативы' if x == 'Сельскохозяйственные потребительские обслуживающие кооперативы' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Сельскохозяйственные потребительские кооперативы' if x == 'Сельскохозяйственные потребительские перерабатывающие кооперативы' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Сельскохозяйственные потребительские кооперативы' if x == 'Сельскохозяйственные потребительские снабженческие кооперативы' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Общественные движения' if x == 'Общественное движение' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [88]:
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Общественные фонды' if x == 'ОБЩЕСТВЕННЫЙ ФОНД' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Некомменческие организации' if x == 'НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Некомменческие организации' if x == 'Некоммерческие партнерства' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Кредитные потребительские кооперативы' if x == 'Кредитные потребительские кооперативы граждан' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [90]:
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Общественные организации' if x == 'Общественные учреждения' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Сельскохозяйственные потребительские кооперативы' if x == 'Сельскохозяйственные потребительские сбытовые (торговые) кооперативы' else x)
nko_data['opf_name'] = nko_data['opf_name'].swifter.apply(
    lambda x: 'Товарищества собственников недвижимости' if x == 'Товарищества собственников жилья' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

#### Работа с признаком `minjustForm`

In [93]:
nko_data['minjustForm'] = nko_data['opf_name'].where(
    nko_data['minjustForm'].isna(), nko_data['minjustForm'])

In [98]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Общественная или религиозная организация' if x == 'Общественная организаци' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Общественная или религиозная организация' if x == 'Общественные организации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Общественная или религиозная организация' if x == 'Общественное учреждение' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Общественное движение' if x == 'Общественные движения' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Сельскохозяйственные потребительские кооперативы' if x == 'Сельскохозяйственные потребительские животноводческие кооперативы' else x)


Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [99]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Общественная или религиозная организация' if x == 'Общественная или религиозная организация (объединение)' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Садоводческие, огороднические или дачные некоммерческие организации' if x == 'Садоводческие, огороднические, дачные и иные некоммерческие товарищества' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Общественная или религиозная организация' if x == 'Общественная организация' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [101]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Садоводческие, огороднические или дачные некоммерческие организации' if x == 'Садоводческое, огородническое, дачное некоммерческое партнерство' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Общественный фонд' if x == 'Общественные фонды' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Адвокатские учреждения' if x == 'Коллегия адвокатов' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Адвокатские учреждения' if x == 'Адвокатское бюро' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Общественный фонд' if x == 'Общественные фонды' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [103]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Иные некоммерческие организации' if x == 'Прочая некоммерческая организация' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Ассоциация крестьянских (фермерских) хозяйств' if x == 'Объединения фермерских хозяйств' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Религиозная организация' if x == 'Религиозная организация' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Религиозная организация' if x == 'Религиозная организация' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [105]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Автономная некоммерческая организация' if x == 'Автономные некоммерческие организации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Политическая партия' if x == 'Политические партии' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Религиозная организация' if x == 'Религиозные организации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Объединения юридических лиц' if x == 'Объединения (союз, ассоциация) юридических лиц' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Объединения юридических лиц' if x == 'Объединение юридических лиц (ассоциация и союз)' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [107]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Ассоциация крестьянских (фермерских) хозяйств' if x == 'Объединение (Союз, Ассоциация) крестьянских (фермерских) хозяйств' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'Объединения юридических лиц' if x == 'Объединение юридических лиц (ассоциация и союз)' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [114]:
nko_data['minjustForm'] = nko_data['minjustForm'].apply(lambda x: str(x).lower())

In [117]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческое образовательное учреждение' if x == 'негосударственное некоммерческое образовательное учреждение' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческое образовательное учреждение' if x == 'негосударственное некоммерческое дошкольное образовательное учреждение' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'другая некоммерческая организация' if x == 'прочие некоммерческие организации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'другая некоммерческая организация' if x == 'иные некоммерческие организации, не включенные в другие группировки' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [119]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'адвокатские и нотариальные организации' if x == 'нотариальные палаты' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'адвокатские и нотариальные организации' if x == 'адвокатские палаты' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'адвокатские и нотариальные организации' if x == 'некоммерческое партнёрство - адвокатское бюро' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческие организации' if x == 'некоммерческое предприятие' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческие организации' if x == 'некоммерческое товарищество' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческие организации' if x == 'некоммерческое объединение' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческие организации' if x == 'местное некоммерческое партнерство' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческая общественная организация' if x == 'общественная некоммерческая организация' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [125]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'благотворительные организации и фонды' if x == 'объединения (ассоциации и союзы) благотворительных организаций' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'благотворительные организации и фонды' if x == 'благотворительные учреждения' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'благотворительные организации и фонды' if x == 'благотворительные фонды' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [129]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'садоводческие, огороднические или дачные некоммерческие организации' if x == 'садоводческое, огородническое, дачное некоммерческое объединение' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'садоводческие, огороднические или дачные некоммерческие организации' if x == 'садоводческое некоммерческое товарищество' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'садоводческие, огороднические или дачные некоммерческие организации' if x == 'ассоциации (союзы) садоводческих, огороднических и дачных некоммерческих объединений' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'садоводческие, огороднические или дачные некоммерческие организации' if x == 'садоводческие, огороднические или дачные объединения граждан' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'садоводческие, огороднические или дачные некоммерческие организации' if x == 'садово-огородническое некоммерческое товарищество' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'садоводческие, огороднические или дачные некоммерческие организации' if x == 'садрводческое некоммерческое товарищество' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'садоводческие, огороднические или дачные некоммерческие организации' if x == 'дачное некоммерческое товарищество' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'садоводческие, огороднические или дачные некоммерческие организации' if x == 'некоммерческое садоводческое товарищество' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'садоводческие, огороднические или дачные некоммерческие организации' if x == 'садоводческое некоммерческое объединение' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [135]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'религиозная организация' if x == 'учреждение общественной, религиозной организации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'религиозная организация' if x == 'учреждения религиозных организаций' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [138]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'адвокатские и нотариальные организации' if x == 'адвокатские бюро' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'адвокатские и нотариальные организации' if x == 'нотариальная палата' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'адвокатские и нотариальные организации' if x == 'адвокатские палаты субъектов рф' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'адвокатские и нотариальные организации' if x == 'коллегии адвокатов' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'адвокатские и нотариальные организации' if x == 'объединения адвокатов' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'адвокатские и нотариальные организации' if x == 'адвокатские учреждения' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [142]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческие организации и фонды' if x == 'некоммерческий фонд' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческие организации и фонды' if x == 'некоммерческое партнерство' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческие организации и фонды' if x == 'некомменческие организации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческие организации и фонды' if x == 'некоммерческое учреждение' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческие организации и фонды' if x == 'некоммерческай организация' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [147]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'сельскохозяйственные организации' if x == 'сельскохозяйственные потребительские кооперативы' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'сельскохозяйственные организации' if x == 'ассоциация крестьянских (фермерских) хозяйств' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'сельскохозяйственные организации' if x == 'сельскохозяйственные потребительские огороднические кооперативы' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'сельскохозяйственные организации' if x == 'сельскохозяйственные потребительские садоводческие кооперативы' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'сельскохозяйственные организации' if x == 'объединения (союзы) крестьянских (фермерских) хозяйств' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [151]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'объединения юридических лиц' if x == 'юридическое лицо, являющееся некоммерческой организацией' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'объединения юридических лиц' if x == 'юридические лица, являющиеся некоммерческими организациями, не включенные в другие группировки' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [153]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'государственные' if x == 'государственная корпорация' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'государственные' if x == 'государственные компании' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'государственные' if x == 'государственно-общественное объединение' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'государственные' if x == 'государственные корпорации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'государственные' if x == 'общественно-государственное общественное объединение' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [156]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'образовательные' if x == 'некоммерческое образовательное учреждение' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'образовательные' if x == 'учреждения академий наук' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'образовательные' if x == 'автономная некоммерческая образовательная организация' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'образовательные' if x == 'совет муниципальных образований' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'образовательные' if x == 'советы муниципальных образований субъектов российской федерации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'образовательные' if x == 'совет муниципальных образований' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [160]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'казачьи объединения' if x == 'казачье общество' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'казачьи объединения' if x == 'казачьи общества, внесенные в государственный реестр казачьих обществ в российской федерации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'казачьи объединения' if x == 'казачьи общества' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [163]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'ассоциации (союзы)' if x == 'союз' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'ассоциации (союзы)' if x == 'союзы (ассоциации) общественных объединений' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'ассоциации (союзы)' if x == 'союзы потребительских обществ' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'ассоциации (союзы)' if x == 'союз потребительских обществ' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'ассоциации (союзы)' if x == 'союз (ассоциация) общественных объединений' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'ассоциации (союзы)' if x == 'ассоциация экономического развития' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [167]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'недвижимость' if x == 'гаражные и гаражно-строительные кооперативы' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'недвижимость' if x == 'товарищества собственников недвижимости' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'недвижимость' if x == 'жилищные или жилищно-строительные кооперативы' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'недвижимость' if x == 'товарищество собственников жилья' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'недвижимость' if x == 'жилищные накопительные кооперативы' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'недвижимость' if x == 'жилищные накопительные кооперативы' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [172]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'иные некоммерческие организации' if x == 'другие некоммерческие организации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'иные некоммерческие организации' if x == 'другая некоммерческая организация' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'кредитные' if x == 'кредитные потребительские кооперативы' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'кредитные' if x == 'кредитные кооперативы второго уровня' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'кредитные' if x == 'союзы (ассоциации) кредитных кооперативов' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [175]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'профессиональные и профсоюзные организации' if x == 'профессиональный союз' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'профессиональные и профсоюзные организации' if x == 'профсоюзные организации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'профессиональные и профсоюзные организации' if x == 'объединение работодателей' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'профессиональные и профсоюзные организации' if x == 'объединения работодателей' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'объединение предприятий' if x == 'объединение предприятий в форме акционерного общества закрытого типа' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'объединение предприятий' if x == 'объединение предприятий в форме полного товарищества' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'объединение предприятий' if x == 'объединение предприятий в форме акционерного общества открытого типа' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [180]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'потребительские организации' if x == 'учреждение потребительской кооперации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'торгово-промышленная палата' if x == 'торгово-промышленные палаты' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'ассоциации (союзы)' if x == 'союзы (ассоциации) кооперативов' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'ассоциации (союзы)' if x == 'ассоциации (союзы) экономического взаимодействия субъектов российской федерации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'ассоциации (союзы)' if x == 'ассоциация' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [186]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'культура' if x == 'национально-культурная автономия' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'культура' if x == 'орган общественной самодеятельности' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'культура' if x == 'автономная некоммерческая просветительская организация' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'фонды' if x == 'фонд' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'фонды' if x == 'фонд' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'малочисленные народы' if x == 'общины коренных малочисленных народов' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'малочисленные народы' if x == 'союзы (ассоциации) общин малочисленных народов' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'малочисленные народы' if x == 'общины коренных малочисленных народов российской федерации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'фонды' if x == 'экологические фонды' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'малочисленные народы' if x == 'община малочисленных народов' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [188]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'садоводческие, огороднические или дачные некоммерческие организации' if x == 'дачное некоммерческое партнерство' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'культура' if x == 'органы общественной самодеятельности' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'пенсионный фонд' if x == 'негосударственный пенсионный фонд' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'пенсионный фонд' if x == 'негосударственные пенсионные фонды' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'адвокатские и нотариальные организации' if x == 'публично-правовые компании' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'учреждение' if x == 'учреждения' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'государственные' if x == 'казенные учреждения' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [190]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческие организации и фонды' if x == 'организация' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческие организации и фонды' if x == 'некоммерческие организации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческие организации и фонды' if x == 'отделения иностранных некоммерческих неправительственных организаций' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческие организации и фонды' if x == 'негосударственная некоммерческая организация' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'некоммерческие организации и фонды' if x == 'саморегулируемые организации' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'общественная или религиозная организация' if x == 'некоммерческая общественная организация' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'общественное движение' if x == 'движение' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [192]:
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'учреждение' if x == 'автономные учреждения' else x)
nko_data['minjustForm'] = nko_data['minjustForm'].swifter.apply(
    lambda x: 'территориальное общественное самоуправление' if x == 'территориальные общественные самоуправления' else x)

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/747034 [00:00<?, ?it/s]

In [194]:
# Сохраняем полученные данные в файл
nko_data.to_csv('data/nko_data_proba.csv', index=False)